# Write your first Qiskit Serverless program

<LegacyContent>
<Admonition type="note">
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/serverless-first-program)
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note">
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/serverless-first-program)
</Admonition>
</CloudContent>

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=1.3.1
qiskit-ibm-runtime~=0.34.0
qiskit-aer~=0.15.1
qiskit-serverless~=0.18.1
qiskit-ibm-catalog~=0.2
qiskit-addon-sqd~=0.8.1
qiskit-addon-utils~=0.1.0
qiskit-addon-mpf~=0.2.0
qiskit-addon-aqc-tensor~=0.1.2
qiskit-addon-obp~=0.1.0
scipy~=1.15.0
pyscf~=2.8.0
```
</details>

This example demonstrates how to use `qiskit-serverless` tools to create a parallel transpilation program, and then implement `qiskit-ibm-catalog` to deploy your program to IBM Quantum Platform to use as a reusable remote service.

## Example: remote transpilation with Qiskit Serverless

Start with the following example that transpiles a `circuit` against a given `backend` and target `optimization_level`, and gradually add more elements to deploy your workload to Qiskit Serverless.

Put the following code cell in the file `./source_files/transpile_remote.py`. This file is the program to upload to Qiskit Serverless.

In [1]:
# This cell is hidden from users, it just creates a new folder
from pathlib import Path

Path("./source_files").mkdir(exist_ok=True)

In [2]:
%%writefile ./source_files/transpile_remote.py

from qiskit.transpiler import generate_preset_pass_manager

def transpile_remote(circuit, optimization_level, backend):
    """Transpiles an abstract circuit into an ISA circuit for a given backend."""
    pass_manager = generate_preset_pass_manager(
        optimization_level=optimization_level,
		backend=backend
    )
    isa_circuit = pass_manager.run(circuit)
    return isa_circuit

Writing ./source_files/transpile_remote.py


## Set up your files

Qiskit Serverless requires setting up your workload’s `.py` files into a dedicated directory. The following structure is an example of good practice:

```text
serverless_program
├── program_uploader.ipynb
└── source_files
    ├── transpile_remote.py
    └── *.py
```

Serverless uploads the contents of `source_files` to run remotely. Once these are set up, you can adjust `transpile_remote.py` to fetch inputs and return outputs.

### Get program arguments

Your initial `transpile_remote.py` has three inputs: `circuits`, `backend_name`, and `optimization_level`. Serverless is currently limited to only accept serializable inputs and outputs. For this reason, you cannot pass in `backend` directly, so use `backend_name` as a string instead.

In [3]:
%%writefile --append ./source_files/transpile_remote.py

from qiskit_serverless import get_arguments, save_result, distribute_task, get

# Get program arguments
arguments = get_arguments()
circuits = arguments.get("circuits")
backend_name = arguments.get("backend_name")
optimization_level = arguments.get("optimization_level")

Appending to ./source_files/transpile_remote.py


<LegacyContent>
At this point, you can get your backend with `QiskitRuntimeService` and add your existing program with the following code. The following code requires that you have already [saved your credentials](/guides/setup-channel#iqp).
</LegacyContent>
<CloudContent>
At this point, you can get your backend with `QiskitRuntimeService` and add your existing program with the following code. The following code requires that you have already [saved your credentials](/guides/cloud-setup).
</CloudContent>

In [4]:
%%writefile --append ./source_files/transpile_remote.py

from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend(backend_name)

Appending to ./source_files/transpile_remote.py


Finally, you can run `transpile_remote()` across all `circuits` passed in, and return the `transpiled_circuits` as a result:

In [5]:
%%writefile --append ./source_files/transpile_remote.py

results = [
    transpile_remote(circuit, 1, backend)
    for circuit in circuits
]

save_result({
    "transpiled_circuits": results
})

Appending to ./source_files/transpile_remote.py


## Deploy to IBM Quantum Platform

The previous section created a program to be run remotely. The code cells in this section upload that program to Qiskit Serverless.

<LegacyContent>
Use `qiskit-ibm-catalog` to authenticate to `QiskitServerless` with your API key, which you can find in your [IBM Quantum account](https://quantum.ibm.com/account), and upload the program.

You can use `save_account()` to save your credentials (See the "Authenticate to the service" step in the [Set up to use IBM Quantum Platform](./setup-channel#set-up-to-use-ibm-quantum-platform) section). Note that this writes your credentials to the same file as [`QiskitRuntimeService.save_account()`](/api/qiskit-ibm-runtime/qiskit-runtime-service#save_account).
</LegacyContent>
<CloudContent>
Use `qiskit-ibm-catalog` to authenticate to `QiskitServerless` with your API key, which you can find on the [IBM Quantum dashboard](https://quantum.cloud.ibm.com), and upload the program.

You can use `save_account()` to save your credentials (See the [Set up to use IBM Cloud](/guides/cloud-setup#cloud-save) section). Note that this writes your credentials to the same file as [`QiskitRuntimeService.save_account()`](/api/qiskit-ibm-runtime/qiskit-runtime-service#save_account).
</CloudContent>

In [6]:
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction

# Authenticate to the remote cluster and submit the pattern for remote execution
serverless = QiskitServerless()

Qiskit Serverless compresses the contents of `working_dir` (in this case, `source_files`) into a `tar`, which is uploaded and cleaned up after. The `entrypoint` identifies the main program executable for Qiskit Serverless to run. Additionally, if your program has custom `pip` dependencies, you can add them to a `dependencies` array:

In [7]:
transpile_remote_demo = QiskitFunction(
    title="transpile_remote_serverless",
    entrypoint="transpile_remote.py",
    working_dir="./source_files/",
)

In [8]:
serverless.upload(transpile_remote_demo)

QiskitFunction(transpile_remote_serverless)

To check if it successfully uploaded, use `serverless.list()`:

In [9]:
# Get program from serverless.list() that matches the title of the one we uploaded
next(
    program
    for program in serverless.list()
    if program.title == "transpile_remote_serverless"
)

QiskitFunction(transpile_remote_serverless)

In [10]:
# This cell is hidden from users, it checks the program uploaded correctly
assert _.title == "transpile_remote_serverless"  # noqa: F821

In [11]:
# This cell is hidden from users, it checks the program executes correctly
from time import sleep
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
transpile_remote_serverless = serverless.load("transpile_remote_serverless")
job = transpile_remote_serverless.run(
    circuits=[qc],
    backend="ibm_sherbrooke",
    optimization_level=1,
)
while True:
    sleep(5)
    status = job.status()
    if status not in ["QUEUED", "INITIALIZING", "RUNNING", "DONE"]:
        raise Exception(
            f"Unexpected job status: '{status}'\n"
            + "Here are the logs:\n"
            + job.logs()
        )
    if status == "DONE":
        break

## Next steps

<Admonition type="info" title="Recommendations">

- Learn how to pass inputs and run your program remotely in the [Run your first Qiskit Serverless workload remotely](./serverless-run-first-workload) topic.

</Admonition>